In [ ]:
import random
from time import time

from matplotlib import pyplot as plt
from mpl_toolkits import axes_grid1
import numpy as np
from IPython.display import Markdown, display
import tensorflow as tf
from tensorflow import keras

import gc
import os

from tabulate import tabulate

# install TF similarity if needed
try:
    import tensorflow_similarity as tfsim  # main package
except ModuleNotFoundError:
    !pip install tensorflow_similarity
    import tensorflow_similarity as tfsim

# INFO messages are not printed.
# This must be run before loading other modules.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"


tfsim.utils.tf_cap_memory()

# Clear out any old model state.
gc.collect()
tf.keras.backend.clear_session()

print("TensorFlow:", tf.__version__)
print("TensorFlow Similarity:", tfsim.__version__)

In [ ]:
from tensorflow.keras import layers
from tensorflow_similarity.layers import MetricEmbedding
from tensorflow_similarity.models import SimilarityModel


# building model
model = tfsim.architectures.EfficientNetSim(
    input_shape= (600, 600, 3), 
    embedding_size = 512, #Max is 512
    variant = "B7",
    weights = "imagenet",
    trainable = "partial", #train last 3 layers
    l2_norm = True,
    include_top = True,
    pooling="max",    # Can change to use `gem` -> GeneralizedMeanPooling2D
#    gem_p=3.0,        # Increase the contrast between activations in the feature map.
)

model.summary()

#ref : https://github.com/tensorflow/similarity/blob/master/api/TFSimilarity/architectures/EfficientNetSim.md

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
batch_size = 5

train_generator = train_datagen.flow_from_directory(
    '../input/kaprao-horapa-train-70',
    target_size=(600, 600),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = valid_datagen.flow_from_directory(
    '../input/kaprao-horapa-val-10',
    target_size=(600, 600),
    batch_size=batch_size,
    class_mode='sparse')

In [ ]:
epochs = 30
learning_rate = 0.0001
gamma = 256

#steps_per_epoch=train_generator.samples/train_generator.batch_size
steps_per_epoch = int(70/batch_size)
#validation_steps=validation_generator.samples/validation_generator.batch_size
val_steps = int(10/batch_size)

# init similarity loss
loss = tfsim.losses.CircleLoss(gamma=gamma)

# compiling and training
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate), 
    loss=loss
)

#callbacks = [
#    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5, verbose=1, mode ='min')
#]

start_train = time()

history = model.fit(
    train_generator, 
    epochs=epochs, 
    validation_data=validation_generator, 
    steps_per_epoch=steps_per_epoch, 
    validation_steps=val_steps,
#    callbacks=callbacks
)

end_train = time()

hours, rem = divmod(end_train-start_train, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(["loss", "val_loss"])
plt.legend(["loss"])
plt.title(f"Loss: {loss.name} - LR: {learning_rate}")
plt.show()

model.save('Kaprao_Horapa_model.h5')